# (24) jobs: beta+conv

**Motivation**: Fit conv models, while changing $\beta$ using the ```vH16``` dataset. Host: ```mach```. <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from figures.fighelper import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Setup

In [2]:
from analysis.helper import job_runner_script


def _cleanup(path, host=None):
    for f in os.listdir(path):
        cond = f.endswith('.txt')
        if host is not None:
            cond = cond and host in f
        if cond:
            os.remove(pjoin(path, f))


def _name(host, gpu_i, fit_i):
    return f"{host}-cuda{gpu_i}-fit{fit_i}"

In [3]:
save_dir = 'Dropbox/git/_PoissonVAE/scripts'
save_dir = pjoin(os.environ['HOME'], save_dir)
os.makedirs(save_dir, exist_ok=True)

# delete existing job runners?
_cleanup(save_dir, None)

print(sorted(os.listdir(save_dir)))

['fit_vae.sh', 'kill_screens.sh', 'resume_fit.sh', 'run_sessions.sh', 'test_tqdm.py', 'test_tqdm.sh']

## Host: mach

In [4]:
host = 'mach'
_cleanup(save_dir, host)

scripts_mach = collections.defaultdict(list)
tot = 0

### Create Fits

In [5]:
n_seeds = 5
seeds = range(1, n_seeds + 1)

betas = [0.01, 0.2, 0.4, 0.6, 0.8, 1.0, 1.5, 4.0]
archi = ['lin|lin', 'conv+b|lin']
t_stop = [0.05, 0.1]

In [6]:
combos_outer = itertools.product(
    archi, t_stop)

for gpu_i, (a, temp) in enumerate(combos_outer):
    combos_inner = itertools.product(
        seeds, betas)
    for s, beta in combos_inner:
        # make arg
        cm = f"soft+t{temp:0.3g}+b{beta:0.2g}"
        arg = ' '.join([
            f"--kl_beta {beta}",
            f"--temp_stop {temp}",
            f"--comment {cm}",
        ])
        kws = dict(
            dataset='vH16',
            model='poisson',
            device=gpu_i,
            archi=a,
            seed=s,
            args=arg,
        )
        scripts_mach[gpu_i].append(job_runner_script(**kws))
        tot += 1

In [7]:
print(tot)

160

In [8]:
scripts_mach = dict(scripts_mach)
print({k: len(v) for k, v in scripts_mach.items()})

{0: 40, 1: 40, 2: 40, 3: 40}

### Save

In [9]:
n_fits = 8

for gpu_i, scripts in scripts_mach.items():
    scripts_divided = divide_list(scripts, n_fits)
    for fit_i, s in enumerate(scripts_divided):
        combined = ' && '.join(s)
        save_obj(
            obj=combined,
            file_name=_name(host, gpu_i, fit_i),
            save_dir=save_dir,
            mode='txt',
        )

[PROGRESS] 'mach-cuda0-fit0.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

...

[PROGRESS] 'mach-cuda3-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit5.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit6.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit7.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

### Print one to check

In [10]:
print(combined.replace('&& ', '&& \n'))

./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 5 --kl_beta 0.6 --temp_stop 0.1 --comment soft+t0.1+b0.6 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 5 --kl_beta 0.8 --temp_stop 0.1 --comment soft+t0.1+b0.8 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 5 --kl_beta 1.0 --temp_stop 0.1 --comment soft+t0.1+b1 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 5 --kl_beta 1.5 --temp_stop 0.1 --comment soft+t0.1+b1.5 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 5 --kl_beta 4.0 --temp_stop 0.1 --comment soft+t0.1+b4

In [11]:
print(scripts_mach)

{
    0: [
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --kl_beta 0.01 --temp_stop 0.05 --comment 
soft+t0.05+b0.01",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --kl_beta 0.2 --temp_stop 0.05 --comment 
soft+t0.05+b0.2",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --kl_beta 0.4 --temp_stop 0.05 --comment 
soft+t0.05+b0.4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --kl_beta 0.6 --temp_stop 0.05 --comment 
soft+t0.05+b0.6",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --kl_beta 0.8 --temp_stop 0.05 --comment 
soft+t0.05+b0.8",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --kl_beta 1.0 --temp_stop 0.05 --comment 
soft+t0.05+b1",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --kl_beta 1.5 --temp_stop 0.05 --comment 
soft+t0.05+b1.5",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --kl_beta 4.0 --temp_stop 0.05 --comment 
soft+t0.05+b4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --kl_beta 0.01 --temp_stop 0.05 --comment 
soft+t0.05+b0.01",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --kl_beta 0.2 --temp_stop 0.05 --comment 
soft+t0.05+b0.2",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --kl_beta 0.4 --temp_stop 0.05 --comment 
soft+t0.05+b0.4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --kl_beta 0.6 --temp_stop 0.05 --comment 
soft+t0.05+b0.6",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --kl_beta 0.8 --temp_stop 0.05 --comment 
soft+t0.05+b0.8",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --kl_beta 1.0 --temp_stop 0.05 --comment 
soft+t0.05+b1",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --kl_beta 1.5 --temp_stop 0.05 --comment 
soft+t0.05+b1.5",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --kl_beta 4.0 --temp_stop 0.05 --comment 
soft+t0.05+b4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --kl_beta 0.01 --temp_stop 0.05 --comment 
soft+t0.05+b0.01",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --kl_beta 0.2 --temp_stop 0.05 --comment 
soft+t0.05+b0.2",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --kl_beta 0.4 --temp_stop 0.05 --comment 
soft+t0.05+b0.4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --kl_beta 0.6 --temp_stop 0.05 --comment 
soft+t0.05+b0.6",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --kl_beta 0.8 --temp_stop 0.05 --comment 
soft+t0.05+b0.8",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --kl_beta 1.0 --temp_stop 0.05 --comment 
soft+t0.05+b1",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --kl_beta 1.5 --temp_stop 0.05 --comment 
soft+t0.05+b1.5",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --kl_beta 4.0 --temp_stop 0.05 --comment 
soft+t0.05+b4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 4 --kl_beta 0.01 --temp_stop 0.05 --comment 
soft+t0.05+b0.01",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 4 --kl_beta 0.2 --temp_stop 0.05 --comment 
soft+t0.05+b0.2",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 4 --kl_beta 0.4 --temp_stop 0.05 --comment 
soft+t0.05+b0.4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 4 --kl_beta 0.6 --temp_stop 0.05 --comment 
soft+t0.05+b0.6",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 4 --kl_beta 0.8 --temp_stop 0.05 --comment 
soft+t0.05+b0.8",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 4 --kl_beta 1.0 --temp_stop 0.05 --comment 
soft+t0.05+b1",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 4 --kl_beta 1.5 --temp_stop 0.05 --comment 
soft+t0.05+b1.5",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 4 --kl_beta 4.0 --temp_stop 0.05 --comment 
soft+t0.05+b4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 5 --kl_beta 0.01 --temp_stop 0.05

In [12]:
print(scripts_divided)

[
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --kl_beta 0.01 --temp_stop 0.1 --comment 
soft+t0.1+b0.01",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --kl_beta 0.2 --temp_stop 0.1 --comment 
soft+t0.1+b0.2",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --kl_beta 0.4 --temp_stop 0.1 --comment 
soft+t0.1+b0.4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --kl_beta 0.6 --temp_stop 0.1 --comment 
soft+t0.1+b0.6",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --kl_beta 0.8 --temp_stop 0.1 --comment 
soft+t0.1+b0.8"
    ],
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --kl_beta 1.0 --temp_stop 0.1 --comment 
soft+t0.1+b1",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --kl_beta 1.5 --temp_stop 0.1 --comment 
soft+t0.1+b1.5",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --kl_beta 4.0 --temp_stop 0.1 --comment 
soft+t0.1+b4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --kl_beta 0.01 --temp_stop 0.1 --comment 
soft+t0.1+b0.01",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --kl_beta 0.2 --temp_stop 0.1 --comment 
soft+t0.1+b0.2"
    ],
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --kl_beta 0.4 --temp_stop 0.1 --comment 
soft+t0.1+b0.4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --kl_beta 0.6 --temp_stop 0.1 --comment 
soft+t0.1+b0.6",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --kl_beta 0.8 --temp_stop 0.1 --comment 
soft+t0.1+b0.8",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --kl_beta 1.0 --temp_stop 0.1 --comment 
soft+t0.1+b1",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --kl_beta 1.5 --temp_stop 0.1 --comment 
soft+t0.1+b1.5"
    ],
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --kl_beta 4.0 --temp_stop 0.1 --comment 
soft+t0.1+b4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --kl_beta 0.01 --temp_stop 0.1 --comment 
soft+t0.1+b0.01",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --kl_beta 0.2 --temp_stop 0.1 --comment 
soft+t0.1+b0.2",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --kl_beta 0.4 --temp_stop 0.1 --comment 
soft+t0.1+b0.4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --kl_beta 0.6 --temp_stop 0.1 --comment 
soft+t0.1+b0.6"
    ],
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --kl_beta 0.8 --temp_stop 0.1 --comment 
soft+t0.1+b0.8",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --kl_beta 1.0 --temp_stop 0.1 --comment 
soft+t0.1+b1",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --kl_beta 1.5 --temp_stop 0.1 --comment 
soft+t0.1+b1.5",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --kl_beta 4.0 --temp_stop 0.1 --comment 
soft+t0.1+b4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 4 --kl_beta 0.01 --temp_stop 0.1 --comment 
soft+t0.1+b0.01"
    ],
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 4 --kl_beta 0.2 --temp_stop 0.1 --comment 
soft+t0.1+b0.2",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 4 --kl_beta 0.4 --temp_stop 0.1 --comment 
soft+t0.1+b0.4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 4 --kl_beta 0.6 --temp_stop 0.1 --comment 
soft+t0.1+b0.6",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 4 --kl_beta 0.8 --temp_stop 0.1 --comment 
soft+t0.1+b0.8",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 4 --kl_beta 1.0 --temp_stop 0.1 --comment 
soft+t0.1+b1"
    ],
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 4 --kl_beta 1.5 --temp_stop 0.1 --comment 
soft+t0.1+b1.5",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 4 --kl_beta 4.0 --temp_stop 0.1 --comment 
soft+t0